# Imports

In [30]:
import pandas as pd
import numpy as np

from dotenv import load_dotenv
import os
from bs4 import BeautifulSoup
import requests

# Constants

In [31]:
dotenv_path = ("/Users/maukanmir/Documents/Machine-Learning/Web-Scraping-Code/Player-Salaries/dot.env")
load_dotenv(dotenv_path)

DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
TABLE_NAME = "player_stats"

# Functions

In [32]:
def extract_nba_stats(html_content, year):
  
  soup = BeautifulSoup(html_content, 'html.parser')
  master_df = []
  tds = soup.find_all("td")
  player_stats = [td.text.strip() for td in tds]
  player_columns = ["Rank", "Name", "Team", "GP", "MPG", "PPG", "FGM", "FGA", "FG%", "3PM", "3PA", "3P%", "FTM", "FTA", "FT%", "ORB", "DRB", "RPG", "APG", "SPG", "BPG", "TOV", "PF"]

  count = 0
  player_dict = {}
  for idx, player_stat in enumerate(player_stats):
    
    if count == 22:
      master_df.append(player_dict)
      player_dict = {}
      count = 0
    else:
      col = player_columns[count]
      player_dict[col] = player_stat
      count +=1
  
  df = pd.DataFrame(master_df)
  df["season"] = year
  return df
  

In [33]:
years = [str(year+1) for year in range(1990, 2024)]
pages = range(1, 4)

In [36]:
all_data= pd.DataFrame()
for year in years:
  for page in pages:
    base_url = f"https://basketball.realgm.com/nba/stats/{year}/Averages/Qualified/points/All/desc/{page}/Regular_Season"
    response = requests.get(base_url)
    try:
      df = extract_nba_stats(response.text, year)
      all_data = pd.concat([all_data, df])
    except Exception as error:
      print(f" THe error is {error}")

In [39]:
all_data.drop("Rank", inplace=True, axis=1)

In [ ]:

# # Create Chrome options if needed (optional)
# opts = Options()
# opts.add_argument("--headless")  # Example option, run Chrome in headless mode

# # Initialize the WebDriver using webdriver-manager
# web_driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)

# # Example usage
# web_driver.get("https://www.nba.com/stats/leaders?SeasonType=Regular+Season&Season=2023-24")
# print(web_driver.title)  # Prints the title of the page